In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 121s 1us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [6]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [7]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 586s 375ms/step - loss: 1.8659 - acc: 0.3126 - val_loss: 1.5949 - val_acc: 0.4219
Epoch 2/100
1563/1563 [==============================] - 564s 361ms/step - loss: 1.5796 - acc: 0.4233 - val_loss: 1.3929 - val_acc: 0.4960
Epoch 3/100
1563/1563 [==============================] - 570s 365ms/step - loss: 1.4719 - acc: 0.4671 - val_loss: 1.2922 - val_acc: 0.5369
Epoch 4/100
1563/1563 [==============================] - 540s 345ms/step - loss: 1.3993 - acc: 0.4986 - val_loss: 1.2387 - val_acc: 0.5627
Epoch 5/100
1563/1563 [==============================] - 527s 337ms/step - loss: 1.3373 - acc: 0.5239 - val_loss: 1.1544 - val_acc: 0.5928
Epoch 6/100
1563/1563 [==============================] - 506s 323ms/step - loss: 1.2787 - acc: 0.5432 - val_loss: 1.1508 - val_acc: 0.5925
Epoch 7/100
1563/1563 [==============================] - 511s 327ms/step - loss: 1.2294 - acc: 0.5647 - val_loss: 1.1523 - val_acc:

1563/1563 [==============================] - 489s 313ms/step - loss: 0.7914 - acc: 0.7341 - val_loss: 0.7238 - val_acc: 0.7555
Epoch 60/100
1563/1563 [==============================] - 499s 320ms/step - loss: 0.7948 - acc: 0.7340 - val_loss: 0.7213 - val_acc: 0.7590
Epoch 61/100
1563/1563 [==============================] - 487s 312ms/step - loss: 0.7963 - acc: 0.7324 - val_loss: 0.6970 - val_acc: 0.7664
Epoch 62/100
1563/1563 [==============================] - 494s 316ms/step - loss: 0.7946 - acc: 0.7357 - val_loss: 0.7175 - val_acc: 0.7597
Epoch 63/100
1563/1563 [==============================] - 490s 313ms/step - loss: 0.7939 - acc: 0.7334 - val_loss: 0.6944 - val_acc: 0.7673
Epoch 64/100
1563/1563 [==============================] - 493s 315ms/step - loss: 0.7909 - acc: 0.7342 - val_loss: 0.7275 - val_acc: 0.7537
Epoch 65/100
1563/1563 [==============================] - 485s 311ms/step - loss: 0.7861 - acc: 0.7367 - val_loss: 0.7470 - val_acc: 0.7506
Epoch 66/100
1563/1563 [=========

In [8]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /Users/i24009/Documents/kaggle/github/deep-learning/experiments-with-keras/saved_models/keras_cifar10_trained_model.h5 
10000/10000 [==============================] - 32s 3ms/step
Test loss: 0.6953575580596923
Test accuracy: 0.7793
